In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col, arrays_zip
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
import pyspark.sql.functions as F
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .appName("data_cleaning") \
    .getOrCreate()

In [3]:
json_data = spark.read.option("multiline","true").json("./Data/JSON/drug-event-0001-of-0005.json")

In [4]:
json_data.printSchema()

root
 |-- meta: struct (nullable = true)
 |    |-- disclaimer: string (nullable = true)
 |    |-- last_updated: string (nullable = true)
 |    |-- license: string (nullable = true)
 |    |-- results: struct (nullable = true)
 |    |    |-- limit: long (nullable = true)
 |    |    |-- skip: long (nullable = true)
 |    |    |-- total: long (nullable = true)
 |    |-- terms: string (nullable = true)
 |-- results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- companynumb: string (nullable = true)
 |    |    |-- duplicate: string (nullable = true)
 |    |    |-- fulfillexpeditecriteria: string (nullable = true)
 |    |    |-- occurcountry: string (nullable = true)
 |    |    |-- patient: struct (nullable = true)
 |    |    |    |-- drug: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- actiondrug: string (nullable = true)
 |    |    |    |    |    |-- activesubstance: struct (nullab

In [5]:
json_data.show()

+--------------------+--------------------+
|                meta|             results|
+--------------------+--------------------+
|{Do not rely on o...|[{US-PFIZER INC-2...|
+--------------------+--------------------+



In [6]:
json_data.select(explode(F.col("results")).alias("exploded_array")).take(1)

[Row(exploded_array=Row(companynumb='US-PFIZER INC-2007080022', duplicate=None, fulfillexpeditecriteria='2', occurcountry=None, patient=Row(drug=[Row(actiondrug=None, activesubstance=None, drugadditional=None, drugadministrationroute=None, drugauthorizationnumb='020998', drugbatchnumb=None, drugcharacterization='1', drugdosageform=None, drugdosagetext=None, drugenddate=None, drugenddateformat=None, drugindication='ARTHRITIS', drugrecurreadministration=None, drugstartdate='20020101', drugstartdateformat='102', drugstructuredosagenumb=None, drugstructuredosageunit=None, drugtreatmentduration='30', drugtreatmentdurationunit='804', medicinalproduct='CELEBREX', openfda=Row(application_number=['NDA020998'], brand_name=['CELEBREX'], generic_name=['CELECOXIB'], manufacturer_name=['PFIZER LABORATORIES DIV PFIZER INC', 'Viatris Specialty LLC'], nui=['N0000000160', 'M0001335', 'N0000175722'], package_ndc=['0025-1515-01', '0025-1520-31', '0025-1520-51', '0025-1520-34', '0025-1525-31', '0025-1525-5

In [7]:
exploded_results = json_data.select(explode(F.col("results")).alias("exploded_results"))

In [8]:
exploded_results.show(5)

+--------------------+
|    exploded_results|
+--------------------+
|{US-PFIZER INC-20...|
|{US-ABBOTT-04P-16...|
|{USA040977859, NU...|
|{2004235828US, NU...|
|{2004-BP-07617BP(...|
+--------------------+
only showing top 5 rows



In [9]:
exploded_results.take(1)

[Row(exploded_results=Row(companynumb='US-PFIZER INC-2007080022', duplicate=None, fulfillexpeditecriteria='2', occurcountry=None, patient=Row(drug=[Row(actiondrug=None, activesubstance=None, drugadditional=None, drugadministrationroute=None, drugauthorizationnumb='020998', drugbatchnumb=None, drugcharacterization='1', drugdosageform=None, drugdosagetext=None, drugenddate=None, drugenddateformat=None, drugindication='ARTHRITIS', drugrecurreadministration=None, drugstartdate='20020101', drugstartdateformat='102', drugstructuredosagenumb=None, drugstructuredosageunit=None, drugtreatmentduration='30', drugtreatmentdurationunit='804', medicinalproduct='CELEBREX', openfda=Row(application_number=['NDA020998'], brand_name=['CELEBREX'], generic_name=['CELECOXIB'], manufacturer_name=['PFIZER LABORATORIES DIV PFIZER INC', 'Viatris Specialty LLC'], nui=['N0000000160', 'M0001335', 'N0000175722'], package_ndc=['0025-1515-01', '0025-1520-31', '0025-1520-51', '0025-1520-34', '0025-1525-31', '0025-1525

In [65]:
all_keys = []

In [66]:
# temp_data.withColumn("keys", F.json_object_keys(temp_data.exploded_array)).show()
keys = exploded_results.select(F.col("exploded_results.*")).columns
keys = ["exploded_results."+str(i) for i in keys]
all_keys.extend(keys)

In [67]:
keys

['exploded_results.companynumb',
 'exploded_results.duplicate',
 'exploded_results.fulfillexpeditecriteria',
 'exploded_results.occurcountry',
 'exploded_results.patient',
 'exploded_results.primarysource',
 'exploded_results.primarysourcecountry',
 'exploded_results.receiptdate',
 'exploded_results.receiptdateformat',
 'exploded_results.receivedate',
 'exploded_results.receivedateformat',
 'exploded_results.receiver',
 'exploded_results.reportduplicate',
 'exploded_results.reporttype',
 'exploded_results.safetyreportid',
 'exploded_results.safetyreportversion',
 'exploded_results.sender',
 'exploded_results.serious',
 'exploded_results.seriousnesscongenitalanomali',
 'exploded_results.seriousnessdeath',
 'exploded_results.seriousnessdisabling',
 'exploded_results.seriousnesshospitalization',
 'exploded_results.seriousnesslifethreatening',
 'exploded_results.seriousnessother',
 'exploded_results.transmissiondate',
 'exploded_results.transmissiondateformat']

In [68]:
patient_keys = exploded_results.select(F.col("exploded_results.patient.*")).columns
patient_keys = ["exploded_results.patient."+str(i) for i in patient_keys]
all_keys.extend(patient_keys)

In [69]:
patient_keys

['exploded_results.patient.drug',
 'exploded_results.patient.patientagegroup',
 'exploded_results.patient.patientdeath',
 'exploded_results.patient.patientonsetage',
 'exploded_results.patient.patientonsetageunit',
 'exploded_results.patient.patientsex',
 'exploded_results.patient.patientweight',
 'exploded_results.patient.reaction',
 'exploded_results.patient.summary']

In [70]:
updated_data = exploded_results.select(all_keys)
updated_data = updated_data.drop(F.col("patient"))

In [71]:
all_keys = updated_data.columns

In [72]:
all_keys

['companynumb',
 'duplicate',
 'fulfillexpeditecriteria',
 'occurcountry',
 'primarysource',
 'primarysourcecountry',
 'receiptdate',
 'receiptdateformat',
 'receivedate',
 'receivedateformat',
 'receiver',
 'reportduplicate',
 'reporttype',
 'safetyreportid',
 'safetyreportversion',
 'sender',
 'serious',
 'seriousnesscongenitalanomali',
 'seriousnessdeath',
 'seriousnessdisabling',
 'seriousnesshospitalization',
 'seriousnesslifethreatening',
 'seriousnessother',
 'transmissiondate',
 'transmissiondateformat',
 'drug',
 'patientagegroup',
 'patientdeath',
 'patientonsetage',
 'patientonsetageunit',
 'patientsex',
 'patientweight',
 'reaction',
 'summary']

In [83]:
updated_data = updated_data.select(all_keys)\
            .withColumn("explode_drug",F.explode(F.col("drug")))

In [116]:
drug_keys = updated_data.select(F.col("explode_drug.*")).columns

In [117]:
drug_keys = ["explode_drug."+i for i in drug_keys]
all_keys.extend(drug_keys)

In [121]:
updated_data.select(all_keys[-2:]).show()

+--------------------+--------------------+
|    medicinalproduct|             openfda|
+--------------------+--------------------+
|            CELEBREX|{[NDA020998], [CE...|
|               TARKA|                NULL|
| HIGH-DOSE DIURETICS|                NULL|
| HYDROCHLOROTHIAZIDE|{[ANDA040780, AND...|
|              FORTEO|{[NDA021318], [FO...|
|              EVISTA|{[NDA020815], [EV...|
|              EVISTA|{[NDA020815], [EV...|
|            CELEBREX|{[NDA020998], [CE...|
|            CELEBREX|{[NDA020998], [CE...|
|             SPIRIVA|{[NDA021395], [SP...|
|           COMBIVENT|                NULL|
|     OXYGEN (OXYGEN)|                NULL|
|SEREVENT DISKUS (...|                NULL|
| HYDROCHLOROTHIAZIDE|{[ANDA040780, AND...|
|              DUONEB|                NULL|
|            CELEBREX|{[NDA020998], [CE...|
|            CELEBREX|{[NDA020998], [CE...|
|             SPIRIVA|{[NDA021395], [SP...|
|           ALBUTEROL|{[ANDA210948, AND...|
| HYDROCHLOROTHIAZIDE|{[ANDA0407

In [122]:
updated_data = updated_data.select(all_keys)\
            .withColumn("explode_reaction",F.explode(F.col("reaction")))

In [130]:
all_keys = updated_data.columns

In [131]:
reaction_keys = updated_data.select(F.col("explode_reaction.*")).columns

In [132]:
reaction_keys = ["explode_reaction."+i for i in reaction_keys]
all_keys.extend(reaction_keys)

In [135]:
all_keys

['companynumb',
 'duplicate',
 'fulfillexpeditecriteria',
 'occurcountry',
 'primarysource',
 'primarysourcecountry',
 'receiptdate',
 'receiptdateformat',
 'receivedate',
 'receivedateformat',
 'receiver',
 'reportduplicate',
 'reporttype',
 'safetyreportid',
 'safetyreportversion',
 'sender',
 'serious',
 'seriousnesscongenitalanomali',
 'seriousnessdeath',
 'seriousnessdisabling',
 'seriousnesshospitalization',
 'seriousnesslifethreatening',
 'seriousnessother',
 'transmissiondate',
 'transmissiondateformat',
 'drug',
 'patientagegroup',
 'patientdeath',
 'patientonsetage',
 'patientonsetageunit',
 'patientsex',
 'patientweight',
 'reaction',
 'summary',
 'actiondrug',
 'activesubstance',
 'drugadditional',
 'drugadministrationroute',
 'drugauthorizationnumb',
 'drugbatchnumb',
 'drugcharacterization',
 'drugdosageform',
 'drugdosagetext',
 'drugenddate',
 'drugenddateformat',
 'drugindication',
 'drugrecurreadministration',
 'drugstartdate',
 'drugstartdateformat',
 'drugstructured

In [133]:
updated_data

DataFrame[companynumb: string, duplicate: string, fulfillexpeditecriteria: string, occurcountry: string, primarysource: struct<qualification:string,reportercountry:string>, primarysourcecountry: string, receiptdate: string, receiptdateformat: string, receivedate: string, receivedateformat: string, receiver: struct<receiverorganization:string,receivertype:string>, reportduplicate: struct<duplicatenumb:string,duplicatesource:string>, reporttype: string, safetyreportid: string, safetyreportversion: string, sender: struct<senderorganization:string,sendertype:string>, serious: string, seriousnesscongenitalanomali: string, seriousnessdeath: string, seriousnessdisabling: string, seriousnesshospitalization: string, seriousnesslifethreatening: string, seriousnessother: string, transmissiondate: string, transmissiondateformat: string, drug: array<struct<actiondrug:string,activesubstance:struct<activesubstancename:string>,drugadditional:string,drugadministrationroute:string,drugauthorizationnumb:

In [138]:
updated_data = updated_data.drop("actiondrug")

In [139]:
updated_data.select(all_keys).show()

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `activesubstance` is ambiguous, could be: [`activesubstance`, `activesubstance`].